In [2]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
ice_cream_train_dir = '../data/food11/train/ice_cream'
ice_cream_test_dir = '../data/food11/test/ice_cream'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(ice_cream_train_dir, fname) for fname in os.listdir(ice_cream_train_dir)]
image_paths += [os.path.join(ice_cream_test_dir, fname) for fname in os.listdir(ice_cream_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 961 images


In [3]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 5

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/ice_cream\1004744.jpg assigned to cluster 2
../data/food11/train/ice_cream\1012277.jpg assigned to cluster 1
../data/food11/train/ice_cream\1031536.jpg assigned to cluster 3
../data/food11/train/ice_cream\1044117.jpg assigned to cluster 4
../data/food11/train/ice_cream\1050427.jpg assigned to cluster 1
../data/food11/train/ice_cream\1052863.jpg assigned to cluster 3
../data/food11/train/ice_cream\1053680.jpg assigned to cluster 3
../data/food11/train/ice_cream\1069788.jpg assigned to cluster 0
../data/food11/train/ice_cream\1075106.jpg assigned to cluster 4
../data/food11/train/ice_cream\1078154.jpg assigned to cluster 4
../data/food11/train/ice_cream\1096437.jpg assigned to cluster 0
../data/food11/train/ice_cream\1100243.jpg assigned to cluster 2
../data/food11/train/ice_cream\1101126.jpg assigned to cluster 1
../data/food11/train/ice_cream\1101478.jpg assigned to cluster 0
../data/food11/train/ice_cream\1106165.jpg assigned to cluster 2
../data/food11/train/ice_

In [4]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(ice_cream_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(ice_cream_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(ice_cream_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(ice_cream_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Fine-grained apple_pie classification
ice_cream_train_gen = ImageDataGenerator(rescale=1./255)
ice_cream_test_gen = ImageDataGenerator(rescale=1./255)

ice_cream_train_generator = ice_cream_train_gen.flow_from_directory(
    ice_cream_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

ice_cream_test_generator = ice_cream_test_gen.flow_from_directory(
    ice_cream_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_ice_cream = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_ice_cream.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_ice_cream = base_model_ice_cream.output
x_ice_cream = GlobalAveragePooling2D()(x_ice_cream)
x_ice_cream = Dense(512, activation='relu')(x_ice_cream)
ice_cream_predictions = Dense(num_clusters, activation='softmax')(x_ice_cream)

# Create and compile the model
ice_cream_model = Model(inputs=base_model_ice_cream.input, outputs=ice_cream_predictions)
ice_cream_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
ice_cream_model.fit(
    ice_cream_train_generator,
    steps_per_epoch=ice_cream_train_generator.samples // 32,
    validation_data=ice_cream_test_generator,
    validation_steps=ice_cream_test_generator.samples // 32,
    epochs=30
)

# Save the model
ice_cream_model.save('fine_grained_ice_cream_classifier.h5')

Found 884 images belonging to 5 classes.
Found 77 images belonging to 5 classes.
Epoch 1/30
27/27 [==============================] - 8s 272ms/step - loss: 1.1398 - accuracy: 0.5587 - val_loss: 0.8013 - val_accuracy: 0.7500
Epoch 2/30
27/27 [==============================] - 7s 241ms/step - loss: 0.5036 - accuracy: 0.8662 - val_loss: 0.5017 - val_accuracy: 0.8594
Epoch 3/30
27/27 [==============================] - 7s 243ms/step - loss: 0.3168 - accuracy: 0.9413 - val_loss: 0.4436 - val_accuracy: 0.7969
Epoch 4/30
27/27 [==============================] - 7s 245ms/step - loss: 0.2272 - accuracy: 0.9648 - val_loss: 0.3239 - val_accuracy: 0.8750
Epoch 5/30
27/27 [==============================] - 7s 246ms/step - loss: 0.1807 - accuracy: 0.9812 - val_loss: 0.3583 - val_accuracy: 0.8438
Epoch 6/30
27/27 [==============================] - 7s 245ms/step - loss: 0.1415 - accuracy: 0.9906 - val_loss: 0.3531 - val_accuracy: 0.8594
Epoch 7/30
27/27 [==============================] - 7s 245ms/step -